In [ ]:
#Run this only if these 2 modules aren't yet installed
# !pip3 install SpeechRecognition pydub

In [ ]:
# !pip install youtube-dl

## 0. Download / extract audio from youtube videos 
... that will serve as examples

In [ ]:
# Exemplo EN
!rm audio.wav
!youtube-dl --extract-audio --audio-format wav -o audioEN_in.wav https://www.youtube.com/watch?v=BHtDLadc7Fo

In [ ]:
import subprocess
command = 'ffmpeg -i /home/jovyan/audioEN_in.wav -ab 160k -ar 44100 -vn /home/jovyan/audioEN.wav'
subprocess.call(command, shell=True)

In [ ]:
# Exemplo PT
!rm audioPT.wav
!youtube-dl --extract-audio --audio-format wav -o audioPT.wav https://www.youtube.com/watch?v=r8y571SrVe4

In [ ]:
command = 'ffmpeg -i /home/jovyan/audioPT_in.wav -ab 160k -ar 44100 -vn /home/jovyan/audioPT.wav'
subprocess.call(command, shell=True)

## 1. Extrair o Audio para um ficheiro "audio.wav"

In [26]:
import subprocess
command = 'ffmpeg -i /home/jovyan/2023-01-11.mp4 -ab 160k -ar 44100 -vn /home/jovyan/audio.wav'
subprocess.call(command, shell=True)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --en

1

## 2. Inicializar o Recognizer

In [34]:
import speech_recognition as sr
# initialize the recognizer
r = sr.Recognizer()

## 3. Função que divide o ficheiro audio em pedaços de alguns segundos 
#### (detecção de frases) e faz o respectivo reconhecimento de cada pedaço
Deste modo não é necessário fazer o upload para a Cloud do Google -- a separação entre pedaços é feita com base nos momentos de pausa entre palavras para não as cortar a meio.

#### Para Português deve-se invocar o Recognizer com a flag language="pt-PT" 
r.recognize_google(audio_data, language="pt-PT") 

In [35]:
# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path, lang):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened, language=lang)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

## 3. Executar a função com o audio extraído do video
A saída é apresentada a seguir a "Todo o Texto"

In [ ]:
# EN
path = "audioEN.wav"
print("\nTexto EN:", get_large_audio_transcription(path, "en-UK"))

result2:
{   'alternative': [   {   'confidence': 0.94262123,
                           'transcript': "my name's Greg and in this video I'm "
                                         'going to teach you how you can '
                                         'evaluate your own level of English '
                                         'according to the common European '
                                         'framework of reference for '
                                         'languages'},
                       {   'transcript': 'my name is Greg and in this video '
                                         "I'm going to teach you how you can "
                                         'evaluate your own level of English '
                                         'according to the common European '
                                         'framework of reference for '
                                         'languages'},
                       {   'transcript': "when is Greg and in

In [33]:
# PT
path = "audioPT.wav"
print("\nTexto PT:", get_large_audio_transcription(path, "pt-PT))

result2:
{   'alternative': [   {   'confidence': 0.78327239,
                           'transcript': 'Hoje vamos falar como devemos '
                                         'planear financeiramente o novo ano '
                                         'Olá a todos o meu nome é Ariana '
                                         'Nunes e este é o canal renda maior '
                                         'este vídeo para por favor deixarem '
                                         'ficar o seu gosto'},
                       {   'transcript': 'Hoje vamos falar como devemos '
                                         'planear financeiramente o novo ano '
                                         'Olá Todos o meu nome é Ariana Nunes '
                                         'e este é o canal renda maior este '
                                         'vídeo para por favor deixarem ficar '
                                         'o seu gosto'},
                       {   'transcript':